# 🌐 Unified Hate Speech Classifier with Language Detection

This notebook provides a unified interface that:
1. **Detects** if input text is Hindi or English using a pretrained language detection model
2. **Routes** Hindi text → Hindi model (XLM-RoBERTa)
3. **Routes** English text → English model (BERT)
4. **Rejects** other languages (including Hinglish, mixed scripts, etc.)

In [7]:
# Install required libraries
!pip install langdetect transformers torch --quiet

In [1]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification, BertConfig
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
import warnings

DetectorFactory.seed = 0

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

: 

In [9]:
class LanguageDetector:
    """
    Language detector that identifies Hindi and English text.
    Uses the langdetect library with additional validation for script detection.
    """
    
    # Unicode ranges for different scripts
    DEVANAGARI_RANGE = (0x0900, 0x097F)  # Hindi script
    ASCII_RANGE = (0x0041, 0x007A)        # Basic Latin letters
    
    def __init__(self):
        self.supported_languages = {'hi': 'hindi', 'en': 'english'}
    
    def _get_script_ratio(self, text):
        """Calculate the ratio of different scripts in the text."""
        devanagari_count = 0
        latin_count = 0
        total_chars = 0
        
        for char in text:
            code = ord(char)
            if char.isalpha():
                total_chars += 1
                if self.DEVANAGARI_RANGE[0] <= code <= self.DEVANAGARI_RANGE[1]:
                    devanagari_count += 1
                elif self.ASCII_RANGE[0] <= code <= self.ASCII_RANGE[1]:
                    latin_count += 1
        
        if total_chars == 0:
            return 0, 0
        
        return devanagari_count / total_chars, latin_count / total_chars
    
    def detect(self, text):
        """
        Detect the language of the input text.
        
        Returns:
            tuple: (language_code, language_name, confidence)
        """
        text = str(text).strip()
        
        if not text:
            return 'unsupported', 'not_supported', 0.0
        
        # First, check script composition
        devanagari_ratio, latin_ratio = self._get_script_ratio(text)
        
        # If text is predominantly Hinglish (mixed scripts), reject it
        if devanagari_ratio > 0.1 and latin_ratio > 0.1:
            return 'unsupported', 'not_supported (hinglish/mixed)', 0.0
        
        # Pure Devanagari script → Hindi
        if devanagari_ratio > 0.7:
            return 'hi', 'hindi', devanagari_ratio
        
        # Pure Latin script → Use langdetect for further classification
        if latin_ratio > 0.7:
            try:
                detected_lang = detect(text)
                if detected_lang == 'en':
                    return 'en', 'english', latin_ratio
                else:
                    return 'unsupported', f'not_supported ({detected_lang})', 0.0
            except LangDetectException:
                return 'unsupported', 'not_supported', 0.0
        
        # Fallback: try langdetect
        try:
            detected_lang = detect(text)
            if detected_lang in self.supported_languages:
                return detected_lang, self.supported_languages[detected_lang], 0.5
            else:
                return 'unsupported', f'not_supported ({detected_lang})', 0.0
        except LangDetectException:
            return 'unsupported', 'not_supported', 0.0

print("✅ LanguageDetector class defined!")

✅ LanguageDetector class defined!


In [10]:
class UnifiedHateSpeechClassifier:
    """
    Unified Hate Speech Classifier that automatically detects language
    and routes to the appropriate model (Hindi or English).
    """
    
    # Fallback HuggingFace model for English hate speech detection
    # This is used when local model weights are not available
    HUGGINGFACE_ENGLISH_MODEL = "Hate-speech-CNERG/bert-base-uncased-hatexplain"
    
    def __init__(self, 
                 english_model_path="./model",
                 hindi_model_path="./hindi_text_classifier"):
        """
        Initialize the unified classifier.
        """
        print("🚀 Initializing Unified Hate Speech Classifier...")
        print("=" * 60)
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"📱 Device: {self.device}")
        
        # Initialize language detector
        self.language_detector = LanguageDetector()
        print("✅ Language Detector initialized")
        
        # Load English model
        self._load_english_model(english_model_path)
        
        # Load Hindi model
        print("\n📚 Loading Hindi model (XLM-RoBERTa)...")
        self.hindi_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
        self.hindi_model = AutoModelForSequenceClassification.from_pretrained(hindi_model_path)
        self.hindi_model.to(self.device)
        self.hindi_model.eval()
        self.hindi_labels = ['hate', 'normal', 'offensive']
        print(f"   Classes: {self.hindi_labels}")
        
        print("\n" + "=" * 60)
        print("✅ Unified Classifier Ready!")
        print("   Supported Languages: Hindi (हिंदी), English")
        print("=" * 60 + "\n")
    
    def _load_english_model(self, english_model_path):
        """Load English model, with fallback to HuggingFace if local weights missing."""
        print("\n📚 Loading English model (BERT)...")
        
        # Check if local model has weights
        local_weights_exist = (
            os.path.exists(os.path.join(english_model_path, "pytorch_model.bin")) or
            os.path.exists(os.path.join(english_model_path, "model.safetensors"))
        )
        
        if local_weights_exist:
            # Load from local path
            print("   Loading from local path...")
            self.english_tokenizer = AutoTokenizer.from_pretrained(english_model_path)
            self.english_model = AutoModelForSequenceClassification.from_pretrained(english_model_path)
            self.english_labels = self.english_model.config.id2label
        else:
            # Fallback to HuggingFace model
            print(f"   ⚠️  Local weights not found at '{english_model_path}'")
            print(f"   📥 Downloading from HuggingFace: {self.HUGGINGFACE_ENGLISH_MODEL}")
            
            self.english_tokenizer = AutoTokenizer.from_pretrained(self.HUGGINGFACE_ENGLISH_MODEL)
            self.english_model = AutoModelForSequenceClassification.from_pretrained(self.HUGGINGFACE_ENGLISH_MODEL)
            self.english_labels = self.english_model.config.id2label
        
        self.english_model.to(self.device)
        self.english_model.eval()
        print(f"   Classes: {list(self.english_labels.values())}")
    
    def _predict_english(self, text):
        """Classify English text using BERT model."""
        inputs = self.english_tokenizer(
            text,
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors='pt'
        )
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.english_model(**inputs)
        
        probabilities = torch.softmax(outputs.logits, dim=-1)
        prediction_idx = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0][prediction_idx].item()
        
        prob_dict = {
            self.english_labels[idx]: round(prob.item(), 4)
            for idx, prob in enumerate(probabilities[0])
        }
        
        return {
            'prediction': self.english_labels[prediction_idx],
            'confidence': round(confidence, 4),
            'probabilities': prob_dict
        }
    
    def _predict_hindi(self, text):
        """Classify Hindi text using XLM-RoBERTa model."""
        inputs = self.hindi_tokenizer(
            text,
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors='pt'
        )
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.hindi_model(**inputs)
            probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
            prediction_idx = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][prediction_idx].item()
        
        prob_dict = {
            self.hindi_labels[idx]: round(prob.item(), 4)
            for idx, prob in enumerate(probabilities[0])
        }
        
        return {
            'prediction': self.hindi_labels[prediction_idx],
            'confidence': round(confidence, 4),
            'probabilities': prob_dict
        }
    
    def classify(self, text):
        """
        Classify input text for hate speech.
        Automatically detects language and routes to appropriate model.
        """
        text = str(text).strip()
        
        if not text:
            return {
                'text': text,
                'detected_language': 'empty',
                'prediction': 'error',
                'confidence': 0.0,
                'probabilities': {},
                'model_used': None,
                'error': 'Empty text provided'
            }
        
        # Detect language
        lang_code, lang_name, lang_confidence = self.language_detector.detect(text)
        
        # Route to appropriate model or return not supported
        if lang_code == 'en':
            result = self._predict_english(text)
            model_used = 'English (BERT)'
        elif lang_code == 'hi':
            result = self._predict_hindi(text)
            model_used = 'Hindi (XLM-RoBERTa)'
        else:
            return {
                'text': text,
                'detected_language': lang_name,
                'prediction': 'NOT SUPPORTED',
                'confidence': 0.0,
                'probabilities': {},
                'model_used': None,
                'message': f'Language "{lang_name}" is not supported. Only Hindi and English are supported.'
            }
        
        return {
            'text': text,
            'detected_language': lang_name,
            'prediction': result['prediction'],
            'confidence': result['confidence'],
            'probabilities': result['probabilities'],
            'model_used': model_used
        }
    
    def classify_batch(self, texts):
        """Classify multiple texts at once."""
        return [self.classify(text) for text in texts]
    
    def analyze(self, texts, verbose=True):
        """
        Analyze multiple texts with formatted output.
        """
        results = []
        
        if verbose:
            print("\n" + "=" * 70)
            print("📊 ANALYSIS RESULTS")
            print("=" * 70)
        
        for i, text in enumerate(texts, 1):
            result = self.classify(text)
            results.append(result)
            
            if verbose:
                print(f"\n{i}. Text: {text[:50]}..." if len(text) > 50 else f"\n{i}. Text: {text}")
                print(f"   🌐 Language: {result['detected_language'].upper()}")
                
                if result['prediction'] == 'NOT SUPPORTED':
                    print(f"   ⚠️  {result['message']}")
                else:
                    pred_emoji = {'hate': '🔴', 'normal': '🟢', 'offensive': '🟡', 'hatespeech': '🔴', 'normal': '🟢', 'offensive': '🟡'}
                    emoji = pred_emoji.get(result['prediction'].lower(), '⚪')
                    print(f"   {emoji} Prediction: {result['prediction'].upper()}")
                    print(f"   📈 Confidence: {result['confidence']:.1%}")
                    print(f"   🤖 Model: {result['model_used']}")
        
        if verbose:
            print("\n" + "=" * 70)
        
        return results

print("✅ UnifiedHateSpeechClassifier class defined!")

✅ UnifiedHateSpeechClassifier class defined!


## 🚀 Initialize the Classifier

In [11]:
# Initialize the unified classifier
# If local English model weights are missing, it will automatically download from HuggingFace
classifier = UnifiedHateSpeechClassifier(
    english_model_path="./model",
    hindi_model_path="./hindi_text_classifier"
)

🚀 Initializing Unified Hate Speech Classifier...
📱 Device: cpu
✅ Language Detector initialized

📚 Loading English model (BERT)...
   ⚠️  Local weights not found at './model'
   📥 Downloading from HuggingFace: Hate-speech-CNERG/bert-base-uncased-hatexplain
   Classes: ['hate speech', 'normal', 'offensive']

📚 Loading Hindi model (XLM-RoBERTa)...
   Classes: ['hate', 'normal', 'offensive']

✅ Unified Classifier Ready!
   Supported Languages: Hindi (हिंदी), English



## 🧪 Test Cases

In [12]:
# Test cases covering different scenarios
test_texts = [
    # ===== ENGLISH TEXTS =====
    "I love spending time with my family and friends",
    "You are such a stupid idiot, go away",
    "All immigrants should be deported immediately",
    
    # ===== HINDI TEXTS =====
    "सबसे अच्छा प्रधानमंत्री नरेंद्र मोदी है",
    "ये कुत्ते की औलाद है सब",
    "आज मौसम बहुत सुहावना है",
    
    # ===== HINGLISH (NOT SUPPORTED) =====
    "Ye bahut bakwaas hai यार",
    "Tu pagal है क्या bro",
    
    # ===== OTHER LANGUAGES (NOT SUPPORTED) =====
    "Je t'aime beaucoup mon ami",  # French
    "Ich liebe dich sehr",          # German
    "こんにちは",                    # Japanese
]

# Analyze all test texts
results = classifier.analyze(test_texts)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.



📊 ANALYSIS RESULTS

1. Text: I love spending time with my family and friends
   🌐 Language: ENGLISH
   🟢 Prediction: NORMAL
   📈 Confidence: 76.2%
   🤖 Model: English (BERT)

2. Text: You are such a stupid idiot, go away
   🌐 Language: ENGLISH
   🟢 Prediction: NORMAL
   📈 Confidence: 67.4%
   🤖 Model: English (BERT)

3. Text: All immigrants should be deported immediately
   🌐 Language: ENGLISH
   🟢 Prediction: NORMAL
   📈 Confidence: 51.9%
   🤖 Model: English (BERT)

4. Text: सबसे अच्छा प्रधानमंत्री नरेंद्र मोदी है
   🌐 Language: HINDI
   🟢 Prediction: NORMAL
   📈 Confidence: 98.7%
   🤖 Model: Hindi (XLM-RoBERTa)

5. Text: ये कुत्ते की औलाद है सब
   🌐 Language: HINDI
   🔴 Prediction: HATE
   📈 Confidence: 84.0%
   🤖 Model: Hindi (XLM-RoBERTa)

6. Text: आज मौसम बहुत सुहावना है
   🌐 Language: HINDI
   🟢 Prediction: NORMAL
   📈 Confidence: 98.3%
   🤖 Model: Hindi (XLM-RoBERTa)

7. Text: Ye bahut bakwaas hai यार
   🌐 Language: NOT_SUPPORTED (HINGLISH/MIXED)
   ⚠️  Language "not_supported 

## 📝 Single Text Classification

In [13]:
# Classify a single text
text = "Enter your text here to classify"
result = classifier.classify(text)

print(f"Text: {result['text']}")
print(f"Language: {result['detected_language']}")
print(f"Prediction: {result['prediction']}")
print(f"Confidence: {result['confidence']:.1%}")
if result.get('model_used'):
    print(f"Model Used: {result['model_used']}")

Text: Enter your text here to classify
Language: english
Prediction: normal
Confidence: 78.0%
Model Used: English (BERT)


## 🎯 Interactive Testing

Run the cell below to test with your own inputs:

In [14]:
# Add your own test texts here
my_texts = [
    "Your English text here",
    "आपका हिंदी टेक्स्ट यहाँ",
]

classifier.analyze(my_texts)


📊 ANALYSIS RESULTS

1. Text: Your English text here
   🌐 Language: ENGLISH
   🟢 Prediction: NORMAL
   📈 Confidence: 74.7%
   🤖 Model: English (BERT)

2. Text: आपका हिंदी टेक्स्ट यहाँ
   🌐 Language: HINDI
   🟢 Prediction: NORMAL
   📈 Confidence: 97.9%
   🤖 Model: Hindi (XLM-RoBERTa)



[{'text': 'Your English text here',
  'detected_language': 'english',
  'prediction': 'normal',
  'confidence': 0.7474,
  'probabilities': {'hate speech': 0.0513,
   'normal': 0.7474,
   'offensive': 0.2013},
  'model_used': 'English (BERT)'},
 {'text': 'आपका हिंदी टेक्स्ट यहाँ',
  'detected_language': 'hindi',
  'prediction': 'normal',
  'confidence': 0.9792,
  'probabilities': {'hate': 0.0142, 'normal': 0.9792, 'offensive': 0.0065},
  'model_used': 'Hindi (XLM-RoBERTa)'}]